In [41]:
import pymysql
import pandas as pd

# 连接数据库
conn = pymysql.connect(host='59.110.49.185',
                       port=3306,
                       user='dmsTest',
                       password='MyIMDBsx123.',
                       database='imdb',
                       charset='utf8')

# 确保查询所有列
query = '''
SELECT id,  popularity,
        adult,  budget, genres,
       release_date,
       revenue, runtime,  status, tagline, vote_average, vote_count
FROM movies_metadata
'''

movies_metadata = pd.read_sql(query, conn)
ratings_small = pd.read_sql('SELECT * FROM ratings_small', conn)
user = pd.read_sql('SELECT * FROM user', conn)
conn.close()

C:\Users\30635\AppData\Local\Temp\ipykernel_23524\3024357561.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movies_metadata = pd.read_sql(query, conn)
C:\Users\30635\AppData\Local\Temp\ipykernel_23524\3024357561.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ratings_small = pd.read_sql('SELECT * FROM ratings_small', conn)
C:\Users\30635\AppData\Local\Temp\ipykernel_23524\3024357561.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user = pd.read_sql('SELECT * FROM user', conn)


In [42]:
import numpy as np

# 创建一个包含所有用户和电影的矩阵
users = user['id']
movies = movies_metadata['id']
user_item_matrix = np.zeros((len(users), len(movies)))

# 构建用户-电影索引字典
user_id_map = {user_id: idx for idx, user_id in enumerate(users)}
movie_id_map = {movie_id: idx for idx, movie_id in enumerate(movies)}

# 使用评分数据填充矩阵
for idx, rating_row in ratings_small.iterrows():
    user_id = rating_row['userId']
    movie_id = rating_row['movieId']
    rating = rating_row['rating']

    if user_id in user_id_map and movie_id in movie_id_map:
        user_idx = user_id_map[user_id]
        movie_idx = movie_id_map[movie_id]
        user_item_matrix[user_idx, movie_idx] = rating

# 将矩阵转换为DataFrame
user_item_matrix = pd.DataFrame(user_item_matrix, index=users, columns=movies)

print(user_item_matrix.shape)
print(user_item_matrix.head())

(600, 40434)
id  2       3       5       6       11      12      13      14      15      \
id                                                                           
1      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5      0.0     4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

id  16      ...  463800  463906  464111  464207  464819  465044  467731  \
id          ...                                                           
1      0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2      0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3      0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4      0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0  

In [43]:
# 将性别转换为哑变量（one-hot encoding）
user_features = pd.get_dummies(user[['age', 'gender']], columns=['gender'])

# 将用户ID作为索引
user_features.index = user['id']

print(user_features.head())

# 转换为df


     age  gender_0  gender_1
id                          
1   20.0     False      True
2   60.0      True     False
3   39.0     False      True
4   51.0      True     False
5   51.0      True     False


In [44]:
# 构建用户-物品评分矩阵
# user_item_matrix = ratings_small.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# print(user_item_matrix.head())


In [45]:
import ast

# 解析 genres 列
def parse_genres(genres_str):
    try:
        genres_list = ast.literal_eval(genres_str)
        return [genre['name'] for genre in genres_list]
    except (ValueError, SyntaxError):
        return []

# 应用解析函数
movies_metadata['genres'] = movies_metadata['genres'].apply(parse_genres)

# 将类型转换为哑变量（one-hot encoding）
item_features = movies_metadata['genres'].str.join('|').str.get_dummies()

item_features=pd.concat([movies_metadata[['popularity', 'vote_average', 'vote_count','adult']],item_features],axis=1)

# 将电影ID作为索引
item_features.index = movies_metadata['id']

print(item_features.head())

    popularity  vote_average  vote_count  adult  Action  Adventure  Animation  \
id                                                                              
2     3.860491           7.1        44.0      0       0          0          0   
3     2.292110           7.1        35.0      0       0          0          0   
5     9.026586           6.5       539.0      0       0          0          0   
6     5.538671           6.4        79.0      0       1          0          0   
11   42.149697           8.1      6778.0      0       1          1          0   

    Comedy  Crime  Documentary  ...  History  Horror  Music  Mystery  Romance  \
id                              ...                                             
2        0      1            0  ...        0       0      0        0        0   
3        1      0            0  ...        0       0      0        0        0   
5        1      1            0  ...        0       0      0        0        0   
6        0      1          

In [47]:
from sklearn.impute import SimpleImputer

# 创建一个简单的填补器，使用均值填补缺失值
imputer = SimpleImputer(strategy='mean')

# 对 item_features 应用填补器
item_features_imputed = imputer.fit_transform(item_features)

# 对 user_features 应用填补器
user_features_imputed = imputer.fit_transform(user_features)

# 将填补后的数据转换为 DataFrame
item_features = pd.DataFrame(item_features_imputed, index=item_features.index, columns=item_features.columns)
user_features = pd.DataFrame(user_features_imputed, index=user_features.index, columns=user_features.columns)

# 检查填补后的数据是否仍然有缺失值
print("item_features_imputed 缺失值数量：", item_features.isnull().sum().sum())
print("user_features_imputed 缺失值数量：", user_features.isnull().sum().sum())

item_features_imputed 缺失值数量： 0
user_features_imputed 缺失值数量： 0


In [50]:
# 检查数据维度
print(user_item_matrix.shape)
print(item_features.shape)
print(user_features.shape)

(600, 40434)
(40434, 24)
(600, 3)


In [49]:
import torch
import pandas as pd
from sklearn.linear_model import LogisticRegression

# 召回阶段
def recall_stage(user_id, user_item_matrix, top_n=100):
    # 基于协同过滤的召回策略
    user_vector = user_item_matrix[user_id]
    scores = user_item_matrix.dot(user_vector)
    top_items = scores.argsort()[-top_n:][::-1]
    return top_items

# 粗排阶段
def rough_ranking(user_id, candidate_items, item_features, top_n=50):
    # 简单的线性模型进行粗排
    model = LogisticRegression()
    model.fit(item_features, candidate_items)
    scores = model.predict_proba(item_features[candidate_items])[:, 1]
    top_items = candidate_items[scores.argsort()[-top_n:][::-1]]
    return top_items

# 精排阶段
def fine_ranking(user_id, candidate_items, item_features, user_features, top_n=10):
    # 使用复杂模型进行精排（例如神经网络）
    model = torch.nn.Sequential(
        torch.nn.Linear(item_features.shape[1] + user_features.shape[1], 128),
        torch.nn.ReLU(),
        torch.nn.Linear(128, 1)
    )
    user_vector = user_features[user_id].repeat(len(candidate_items), 1)
    input_features = torch.cat((item_features[candidate_items], user_vector), dim=1)
    scores = model(input_features)
    top_items = candidate_items[scores.argsort()[-top_n:][::-1]]
    return top_items

# 主推荐流程
def recommend_movies(user_id, user_item_matrix, item_features, user_features, movies_metadata):
    # 召回阶段
    recall_items = recall_stage(user_id, user_item_matrix)

    # 粗排阶段
    rough_items = rough_ranking(user_id, recall_items, item_features)

    # 精排阶段
    final_items = fine_ranking(user_id, rough_items, item_features, user_features)

    # 返回推荐结果
    recommended_movies = movies_metadata[movies_metadata['id'].isin(final_items)]
    return recommended_movies

# 假设已加载user_item_matrix, item_features, user_features, movies_metadata
user_id = 2
recommended_movies = recommend_movies(user_id, user_item_matrix, item_features, user_features, movies_metadata)
print(recommended_movies)


ValueError: matrices are not aligned